In [ ]:
# Imports
import pandas as pd
import numpy as np
import polars as pl
import os
import awpy
from demoparser2 import DemoParser
from awpy import Demo
from pathlib import Path
from awpy.stats import adr
from awpy.stats import kast
from awpy.stats import rating
from awpy.stats import calculate_trades
from supabase import create_client, Client
from dotenv import load_dotenv
from requests import post, get

In [ ]:
folder_path = r'C:\Users\bayli\Documents\CS Demos\test_demos'
file_test = r'C:\Users\bayli\Documents\CS Demos\test_demos\natus-vincere-vs-faze-m1-inferno.dem'

# Creating DataFrames
df_flashes = pd.DataFrame()
df_he = pd.DataFrame()
df_infernos = pd.DataFrame()
df_smoke = pd.DataFrame()
df_kills = pd.DataFrame()
df_rounds = pd.DataFrame()
df_all_first_kills = pd.DataFrame()
df_adr = pd.DataFrame()
df_kast = pd.DataFrame()
df_util_dmg = pd.DataFrame()
team_rounds_won = pd.DataFrame()
players_id = pd.DataFrame()
df_matches = pd.DataFrame(columns=['event_id','match_name'])
i = 1
current_schema = "staging"
event_id = 3

In [ ]:
def add_round_winners(ticks_df, rounds_df):
    ticks_df = ticks_df.to_pandas()
    rounds_df = rounds_df.to_pandas()

    # Makes sure the columns exists
    rounds_df['ct_team_clan_name'] = None
    rounds_df['t_team_clan_name'] = None
    rounds_df['winner_clan_name'] = None
    rounds_df['ct_team_current_equip_value'] = None
    rounds_df['t_team_current_equip_value'] = None
    rounds_df['ct_losing_streak'] = None
    rounds_df['t_losing_streak'] = None

    for idx, row in rounds_df.iterrows():
        freeze_end_tick = row['freeze_end']
        winner = row['winner']

        # Takes all corresponding entries
        first_tick_df = ticks_df[ticks_df['tick'] == freeze_end_tick]

        # Takes the name for every team
        try:
            CT_team = first_tick_df[first_tick_df['side'] == 'ct']['team_clan_name'].iloc[0]
        except IndexError:
            CT_team = None
        
        try:
            T_team = first_tick_df[first_tick_df['side'] == 't']['team_clan_name'].iloc[0]
        except IndexError:
            T_team = None

        # Takes the current equip value for every team
        try:
            CT_team_current_equip_value = first_tick_df[first_tick_df['side'] == 'ct']['current_equip_value'].sum()
        except KeyError:
            CT_team_current_equip_value = None

        try:
            T_team_current_equip_value = first_tick_df[first_tick_df['side'] == 't']['current_equip_value'].sum()
        except KeyError:
            T_team_current_equip_value = None

        # Determines the winner team name
        if winner == 'ct':
            winner_clan = CT_team
        elif winner in ['t', 'TERRORIST']:
            winner_clan = T_team
        else:
            winner_clan = None
            print(f"[!] Round {idx} - winner error: '{winner}'")
            
        # Fill Columns in the DataFrame
        rounds_df.at[idx, 'ct_team_clan_name'] = CT_team
        rounds_df.at[idx, 't_team_clan_name'] = T_team
        rounds_df.at[idx, 'winner_clan_name'] = winner_clan
        rounds_df.at[idx, 'ct_team_current_equip_value'] = CT_team_current_equip_value
        rounds_df.at[idx, 't_team_current_equip_value'] = T_team_current_equip_value


    return rounds_df

def add_losing_streaks(df: pd.DataFrame) -> pd.DataFrame:
    ct_losing_streak = []
    t_losing_streak = []

    ct_streak = 0
    t_streak = 0

    for _, row in df.iterrows():
        ct_team = row['ct_team_clan_name']
        t_team = row['t_team_clan_name']
        winner = row['winner_clan_name']
        
        if winner == ct_team:
            ct_streak = 0
            t_streak += 1
        else:  # winner == t_team
            t_streak = 0
            ct_streak += 1

        ct_losing_streak.append(ct_streak)
        t_losing_streak.append(t_streak)

    df['ct_losing_streak'] = ct_losing_streak
    df['t_losing_streak'] = t_losing_streak

    return df

def add_buy_type(row):

    if row['round_num'] in [1, 13]:
        return "Pistol", "Pistol"

    if row['ct_team_current_equip_value'] < 5000:
        ct_buy_type = "Full Eco"
    elif 5000 <= row['ct_team_current_equip_value'] < 10000:
        ct_buy_type = "Semi-Eco"
    elif 10000 <= row['ct_team_current_equip_value'] < 20000:
        ct_buy_type = "Semi-Buy"
    elif row['ct_team_current_equip_value'] >= 20000:
        ct_buy_type = "Full Buy"
    else:
        ct_buy_type = "Unknown"

    if row['t_team_current_equip_value'] < 5000:
        t_buy_type = "Full Eco"
    elif 5000 <= row['t_team_current_equip_value'] < 10000:
        t_buy_type = "Semi-Eco"
    elif 10000 <= row['t_team_current_equip_value'] < 20000:
        t_buy_type = "Semi-Buy"
    elif row['t_team_current_equip_value'] >= 20000:
        t_buy_type = "Full Buy"
    else:
        t_buy_type = "Unknown"

    return ct_buy_type, t_buy_type

def calculate_advantage_5v4(rounds_df, first_kills_df):

    # Makes sure the columns exists
    rounds_df['advantage_5v4'] = None

    # Checks what team got the first kill
    for idx, row in rounds_df.iterrows():
        round_num = row['round_num']

        # Filters the first kills DataFrame for the current round
        first_kill = first_kills_df[first_kills_df['round_num'] == round_num]

        if not first_kill.empty:
            # Gets the team that made the first kill
            killer_team = first_kill.iloc[0]['attacker_side']

            # Defines the advantage based on the killer team
            if killer_team == 'ct':
                rounds_df.at[idx, 'advantage_5v4'] = 'ct'
            elif killer_team == 't':
                rounds_df.at[idx, 'advantage_5v4'] = 't'

    return rounds_df

def insert_table(df, current_schema, table_name, conflict_cols):
    for row in df.to_dict(orient="records"):
        supabase.schema(current_schema).table(table_name).upsert(row, on_conflict=conflict_cols).execute()

def insert_or_update_player_history(players_df):
    for _, row in players_df.iterrows():
        steam_id = row["steam_id"]
        team_id = row["team_id"]
        
        # Finds out if the players have changed teams
        player_history_data = {
            "steam_id": steam_id,
            "team_id": team_id
        }
        # Updates the player history table with the new data
        supabase.table("player_history").upsert(
            player_history_data, 
            on_conflict=["steam_id", "team_id"]
        ).execute()

def rounds_correction(df: pl.DataFrame) -> pl.DataFrame:
    freeze_end_is_null = df.select(pl.col("freeze_end").first().is_null()).item()

    if freeze_end_is_null:
        df = df.with_columns(
            (pl.col("round_num") - 1).alias("round_num")
        )
        
    return df.filter(pl.col("freeze_end").is_not_null())

def fetch_all_rows(current_schema, table_name, page_size=1000):
    offset = 0
    all_data = []

    while True:
        response = supabase.schema(current_schema).table(table_name).select("*").range(offset, offset + page_size - 1).execute()
        data = response.data
        if not data:
            break
        all_data.extend(data)
        offset += page_size

    return all_data

In [ ]:
# Full Loop
for file_name in os.listdir(folder_path):
    if file_name.endswith('.dem'):

        file_path = os.path.join(folder_path, file_name)
        dem = Demo(file_path)
        dem.parse(player_props=["team_clan_name","total_rounds_played", "current_equip_value", "ct_losing_streak", "t_losing_streak"])

        # Gets all the Players' steam_ids
        this_file_players_id = dem.events.get('player_spawn')
        this_file_players_id = this_file_players_id.with_columns(
            this_file_players_id['user_steamid'].cast(pl.Utf8)
        )
        this_file_players_id = this_file_players_id.to_pandas()
        this_file_players_id = this_file_players_id[['user_steamid', 'user_name']].drop_duplicates()
        players_id = pd.concat([players_id, this_file_players_id], ignore_index=True)
        players_id = players_id[['user_steamid', 'user_name']].drop_duplicates()

        # Grenades Data
        # Makes that the data frame is not empty and that the columns are in the right format
        this_file_flashes = dem.events.get('flashbang_detonate', pl.DataFrame())
        if this_file_flashes is not None and len(this_file_flashes) > 0:
            this_file_flashes = this_file_flashes.with_columns(
                this_file_flashes['user_steamid'].cast(pl.Utf8)
            )
        this_file_he = dem.events.get('hegrenade_detonate', pl.DataFrame())
        if this_file_he is not None and len(this_file_he) > 0:
            this_file_he = this_file_he.with_columns(
                this_file_he['user_steamid'].cast(pl.Utf8)
            )
        this_file_infernos = dem.events.get('inferno_startburn', pl.DataFrame())
        if this_file_infernos is not None and len(this_file_infernos) > 0:
            this_file_infernos = this_file_infernos.with_columns(
                this_file_infernos['user_steamid'].cast(pl.Utf8)
            )
        this_file_smoke = dem.events.get('smokegrenade_detonate', pl.DataFrame())
        if this_file_smoke is not None and len(this_file_smoke) > 0:
            this_file_smoke = this_file_smoke.with_columns(
                this_file_smoke['user_steamid'].cast(pl.Utf8)
            )
        this_file_util_dmg = dem.events.get('player_hurt', pl.DataFrame())
        if this_file_util_dmg is not None and len(this_file_util_dmg) > 0:
            this_file_util_dmg = this_file_util_dmg.with_columns(
                this_file_util_dmg['attacker_steamid'].cast(pl.Utf8)
            )
        util_dmg = this_file_util_dmg.filter(
            (this_file_util_dmg["weapon"] == "hegrenade") |
            (this_file_util_dmg["weapon"] == "molotov")   |
            (this_file_util_dmg["weapon"] == "inferno")
        )
        # Makes sure that the data frames are not empty, converts them to pandas and appends them to the main data frame
        if this_file_flashes is not None and len(this_file_flashes) > 0:
            df_flashes = pd.concat([df_flashes, this_file_flashes.to_pandas()], ignore_index=True)
        if this_file_he is not None and len(this_file_he) > 0:
            df_he = pd.concat([df_he, this_file_he.to_pandas()], ignore_index=True)
        if this_file_infernos is not None and len(this_file_infernos) > 0:
            df_infernos = pd.concat([df_infernos, this_file_infernos.to_pandas()], ignore_index=True)
        if this_file_smoke is not None and len(this_file_smoke) > 0:
            df_smoke = pd.concat([df_smoke, this_file_smoke.to_pandas()], ignore_index=True)
        if this_file_util_dmg is not None and len(this_file_util_dmg) > 0:
            df_util_dmg = pd.concat([df_util_dmg, this_file_util_dmg.to_pandas()], ignore_index=True)

        # Opening Kills Data
        this_file_df_kills = awpy.stats.calculate_trades(demo=dem)
        this_file_df_kills = this_file_df_kills.with_columns(
            this_file_df_kills['attacker_steamid'].cast(pl.Utf8),
            this_file_df_kills['assister_steamid'].cast(pl.Utf8),
            this_file_df_kills['victim_steamid'].cast(pl.Utf8)
        )
        this_file_df_kills = this_file_df_kills.to_pandas()
        first_kills = this_file_df_kills.sort_values(by=['round_num', 'tick'])
        first_kills = first_kills.groupby('round_num').first().reset_index()
        df_all_first_kills = pd.concat([df_all_first_kills, first_kills], ignore_index=True)

        # Creates Match Table
        folder_name = os.path.basename(os.path.dirname(file_path))
        file_name = file_name.replace(f"{folder_name}_", "")
        df_matches = pd.concat([df_matches, pd.DataFrame({'match_name': [file_name], 'event_id': [event_id]})], ignore_index=True)

        # Rounds Data
        this_file_df_ticks = dem.ticks
        this_file_df_rounds = dem.rounds
        this_file_df_rounds = rounds_correction(this_file_df_rounds)
        this_file_df_rounds = add_round_winners(this_file_df_ticks,this_file_df_rounds)
        this_file_df_rounds = add_losing_streaks(this_file_df_rounds)
        this_file_df_rounds[['ct_buy_type', 't_buy_type']] = this_file_df_rounds.apply(add_buy_type, axis=1, result_type='expand')
        first_kills = this_file_df_kills.sort_values(by=['round_num', 'tick'])
        first_kills = first_kills.groupby('round_num').first().reset_index()
        df_all_first_kills = pd.concat([df_all_first_kills, first_kills], ignore_index=True)   

        this_file_df_rounds = calculate_advantage_5v4(this_file_df_rounds, df_all_first_kills)
        this_file_df_rounds['match_name'] = file_name
        df_rounds = pd.concat([df_rounds, this_file_df_rounds], ignore_index=True)
        df_rounds['event_id'] = event_id
        # Creates rounds won columns
        this_file_team_rounds_won = this_file_df_rounds.groupby('winner_clan_name').agg(
            total_rounds_won=('winner_clan_name', 'size'),
            t_rounds_won=('winner', lambda x: (x == 'ct').sum()),
            ct_rounds_won=('winner', lambda x: (x == 't').sum())
        ).reset_index()
        this_file_team_rounds_won.columns = ['team_clan_name', 'total_rounds_won','t_rounds_won', 'ct_rounds_won']
        team_rounds_won = pd.concat([team_rounds_won,this_file_team_rounds_won], ignore_index=True)
        df_kills = pd.concat([df_kills,this_file_df_kills], ignore_index=True)

        # ADR Data
        this_file_adr = awpy.stats.adr(demo=dem)
        this_file_adr = this_file_adr.with_columns(
            this_file_adr['steamid'].cast(pl.Utf8)
        )
        this_file_adr = this_file_adr.to_pandas()
        this_file_adr = this_file_adr.drop(['adr', 'name'], axis=1)
        df_adr = pd.concat([df_adr, this_file_adr], ignore_index=True)
        df_adr = df_adr.groupby(['steamid','side'], as_index=False).sum()
        df_adr = df_adr[df_adr['side'] != 'all']


        # KAST Data
        this_file_kast = awpy.stats.kast(demo=dem)
        this_file_kast = this_file_kast.with_columns(
            this_file_kast['steamid'].cast(pl.Utf8)
        )
        this_file_kast = this_file_kast.to_pandas()
        this_file_kast = this_file_kast.drop(['kast', 'name'], axis=1)
        df_kast = pd.concat([df_kast, this_file_kast], ignore_index=True)
        df_kast = df_kast.groupby(['steamid','side'], as_index=False).sum()
        df_kast = df_kast[df_kast['side'] != 'all']

        print(f"{i}: Processed {file_name}")
        i = i + 1

In [ ]:
dem = Demo(file_test)
dem.parse(player_props=["team_clan_name","total_rounds_played", "current_equip_value", "round_num", "is_alive"])

In [ ]:
dem.rounds

In [ ]:
dem.kills

In [ ]:
ticks = dem.ticks
ticks

In [ ]:
clutches_data = []

# for round_info in dem.rounds.iter_rows(named=True):
#     round_num = round_info['round_num']
#     round_start_tick = round_info['start']
#     round_end_tick = round_info['end']
#     winning_team = round_info['winner']
for round_info in dem.rounds.iter_rows(named=True):
    round_num = 18
    round_start_tick = 175000
    round_end_tick = 183338
    winning_team = round_info['winner']

    # Filtrar ticks para a rodada atual
    round_ticks = dem.ticks.filter(
        (pl.col("tick") >= round_start_tick) & (pl.col("tick") <= round_end_tick)
    )

    # Calcular jogadores vivos por equipe em cada tick
    alive_players_per_tick = round_ticks.group_by(["tick", "side"]).agg(
        pl.col("is_alive").sum().alias("alive_count")
    )

    clutch_active = False
    clutcher_steamid = None
    clutcher_team_num = None
    clutch_start_tick = None
    clutch_kills = 0

    # Iterar pelos ticks para detectar o clutch
    for tick_data in alive_players_per_tick.sort("tick").iter_rows(named=True):
        current_tick = tick_data['tick']
        
        # Obter contagem de jogadores vivos para ambas as equipes neste tick
        team_counts = alive_players_per_tick.filter(pl.col("tick") == current_tick).to_dicts()
        print(team_counts)
        
        team_alive_counts = {item['side']: item['alive_count'] for item in team_counts}
        
        for side, alive_count in team_alive_counts.items():
            if side == "ct":
                opponent_team = "t"
            elif side == "t":
                opponent_team = "ct"
            else:
                opponent_team = None

            opponent_alive_count = team_alive_counts.get(opponent_team, 0)

            if alive_count == 1 and opponent_alive_count >= 2:
                # Condição de clutch detectada
                if not clutch_active:
                    clutch_active = True
                    clutcher_team_num = side
                    clutch_start_tick = current_tick
                    # Encontrar o steamid do clutcher
                    clutcher_info = round_ticks.filter(
                        (pl.col("tick") == current_tick) & 
                        (pl.col("side") == clutcher_team_num) & 
                        (pl.col("is_alive") == True)
                    ).select("steamid").unique().item()
                    clutcher_steamid = clutcher_info
                
                # Se o clutch estiver ativo e o clutcher ainda for o mesmo
                # (para evitar re-detecção de um novo clutch se o jogador morrer e outro entrar em 1vX)
                if clutch_active and clutcher_steamid is not None:
                    # Contar kills durante o período do clutch
                    current_clutch_kills = dem.kills.filter(
                        (pl.col("round_num") == round_num) &
                        (pl.col("tick") >= clutch_start_tick) &
                        (pl.col("tick") <= current_tick) & # Kills até o tick atual
                        (pl.col("attacker_steamid") == clutcher_steamid)
                    ).shape
                    clutch_kills = current_clutch_kills # Atualiza a contagem de kills
            else:
                # Se a condição de clutch não for mais atendida
                if clutch_active:
                    # Clutch terminou
                    clutch_end_tick = current_tick - 1 # O tick anterior foi o último do clutch
                    
                    # Determinar sobrevivência do clutcher
                    clutcher_final_state = round_ticks.filter(
                        (pl.col("tick") == clutch_end_tick) & 
                        (pl.col("steamid") == clutcher_steamid)
                    ).select("is_alive").item()
                    
                    clutcher_survived = clutcher_final_state
                    
                    clutches_data.append({
                        "round_num": round_num,
                        "clutcher_steamid": clutcher_steamid,
                        "clutcher_team_num": clutcher_team_num,
                        "clutch_start_tick": clutch_start_tick,
                        "clutch_end_tick": clutch_end_tick,
                        "clutch_kills": clutch_kills,
                        "clutcher_survived": clutcher_survived
                    })
                    
                    # Resetar variáveis para o próximo clutch
                    clutch_active = False
                    clutcher_steamid = None
                    clutcher_team_num = None
                    clutch_start_tick = None
                    clutch_kills = 0
                    
    # Lidar com clutches que duram até o final da rodada
    if clutch_active:
        clutcher_final_state = round_ticks.filter(
            (pl.col("tick") == round_end_tick) & 
            (pl.col("steamid") == clutcher_steamid)
        ).select("is_alive").item()
        
        clutcher_survived = clutcher_final_state and (
            (clutcher_team_num == 2 and winning_team == "CT") or
            (clutcher_team_num == 3 and winning_team == "T")
        )
        
        clutches_data.append({
            "round_num": round_num,
            "clutcher_steamid": clutcher_steamid,
            "clutcher_team_num": clutcher_team_num,
            "clutch_start_tick": clutch_start_tick,
            "clutch_end_tick": round_end_tick,
            "clutch_kills": clutch_kills,
            "clutcher_survived": clutcher_survived
        })

# Converter resultados para DataFrame (Pandas ou Polars)
# clutches_df = pl.DataFrame(clutches_data) ou pd.DataFrame(clutches_data)

In [ ]:
team_counts

In [ ]:
clutches_data

In [ ]:
clutches_data

In [ ]:
# Rounds Data
this_file_df_ticks = dem.ticks
this_file_df_rounds = dem.rounds
this_file_df_rounds = rounds_correction(this_file_df_rounds)
this_file_df_rounds = add_round_winners(this_file_df_ticks,this_file_df_rounds)
this_file_df_rounds = add_losing_streaks(this_file_df_rounds)
this_file_df_rounds[['ct_buy_type', 't_buy_type']] = this_file_df_rounds.apply(add_buy_type, axis=1, result_type='expand')
this_file_df_rounds